# IMPORTS

[Reference](https://www.kaggle.com/dschettler8845/visual-in-depth-eda-vinbigdata-competition-data)

In [ ]:
# Machine Learning and Data Sciences library
import numpy as np
import pandas as pd
import scipy

# Built-in 
from glob import glob
import IPython
import zipfile
import shutil
import tqdm
import os

# Visualize library
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import cv2

# Presets
FIG_FONT = dict(family='Helvetica, Arial', size=14, color='#7f7f7f')

# Others
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

print('\n...IMPORT COMPLETE...\n')

In [ ]:
# Import color pallete 14 colors for 14 classes
# https://seaborn.pydata.org/tutorial/color_palettes.html
# https://matplotlib.org/3.3.3/tutorials/colors/colormaps.html

color_palette = sns.color_palette("rainbow", 14)

# hex code format
color_palette_hex = color_palette.as_hex()
# (r,b,g) format
# NOTE: make sure the elements in color tuple are int, not numpy.int
color_palette_rbg = []
for color in color_palette:
    color_rgb = tuple(int(c*255) for c in color)
    color_palette_rbg.append(color_rgb)

sns.palplot(color_palette_hex)
# for easy to view, use special color for class "No finding"
color_palette_hex.append("#fae1dd")

# NOTEBOOK SETUP

In [ ]:
# define the root data directory
DATA_DIR = "../input/vinbigdata-chest-xray-abnormalities-detection"

# define the path to the train and test dicom folders
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")

# get all dicom file paths in train and test folders
TRAIN_DICOM_PATHS = [
    os.path.join(TRAIN_DIR, file_name) \
    for file_name in os.listdir(TRAIN_DIR)
]
TEST_DICOM_PATHS = [
    os.path.join(TEST_DIR, file_name) \
    for file_name in os.listdir(TEST_DIR)
]

print(f"\n...The number of training files is {len(TRAIN_DICOM_PATHS)}...")
print(f"...The number of testing files is {len(TEST_DICOM_PATHS)}...")

# define paths to csv files
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
SS_CSV = os.path.join(DATA_DIR, "sample_submission.csv")

# create dataframe from csv files
train_df = pd.read_csv(TRAIN_CSV)
ss_df = pd.read_csv(SS_CSV)

print("\nTRAIN DATAFRAME\n")
display(train_df.head())

print("\nSAMPLE SUBMISSION DATAFRAME\n")
display(ss_df.head())

# EXPLORE CSV DATA

1. **IMAGE_ID EXPLORATION**

In [ ]:
import plotly.express as px

# see number of annotations (bboxes) per an image
# or number of image contain n annotations
fig = px.histogram(train_df['image_id'].value_counts(), opacity=.7,
                   log_y=True, color_discrete_sequence=['salmon'],
                   labels={'value':'# annotations per image'},
                   title="DISTRIBUTION OF NUMBER ANNOTATIONS PER IMAGE/ PATIENT")

fig.update_layout(showlegend=False,
                 xaxis_title="# annotations",
                 yaxis_title="# unique images")
fig.show()

**From the histogram plotted above we can ascertain the following information:**
* An image can be labeled by up to 3 radiologists, so there can be multiple bboxes for the same abnomalities
* Annotations can be included "No finding"
* Images contain at least 3 annotations (~11,000 images)
* Images contain at most 57 annotations (1 image)

2. **CLASS_ID/ CLASS_ID EXPLORATION**

The class_id column indicates the "label (class_name) encoded as number". We would rather work with a numeric labels representation. So we will create a dictionary which allow us to translate numeric labels back into their respective string lables, and vice versa.

In [ ]:
# create a list of class name class name corresponding to class_id order

# get unique class name in df and convert to list
class_name = train_df["class_name"].unique().tolist()
# all class name are in alphabet order except "No finding"
class_name.sort()
# move "No finding" to the end corresponding to class id 14
class_name.append(class_name.pop(class_name.index("No finding")))

print(class_name)

In [ ]:
# create dictionary mappings
classID_to_STR = {i:class_name[i] for i in range(15)}
classSTR_to_ID = {class_name[i]:i for i in range(15)}

display(classID_to_STR)
display(classSTR_to_ID)

In [ ]:
# see number of annotations per class
fig = px.bar(train_df["class_id"].value_counts().sort_index(), opacity=.8,
             log_y=True, color=class_name,
             labels={"value":"# annotation of class"},
             title="NUMBER OF ANNOTATIONS PER CLASS")

fig.update_layout(legend_title=None,
                  xaxis_title="classes",
                  yaxis_title="# annotation")
fig.show()

In [ ]:
# non-log scale y axis version
plt.figure(figsize=(10, 10))
sns.countplot(x="class_id", data=train_df)
plt.title("Class ID Distribution")
plt.show()

3. **RAD_ID EXPLORATION**

In [ ]:
# see number of annotations were labeled by a radiologist
fig = px.bar(train_df["rad_id"].value_counts(), opacity=.8,
             labels={"value":"# annotation", "color":"rad ID"}, 
             color=train_df["rad_id"].value_counts().keys(),
             title="DISTRIBUTION OF # ANNOTATIONS PER RADIOLOGIST")

fig.update_layout(legend_title="RADIOLOGIST ID",
                  xaxis_title="Radiologist ID",
                  yaxis_title="# annotation")
fig.show()

**From the histogram plotted below we can ascertain the following information**

3 of the radiologists (R9, R10, & R8 in that order) are responsible for the vast majority of annotations (~40-50% of all annotations)

In [ ]:
# see the expertise of radiologist
# whether all 17 radiologists can label all 15 distinct objects?
# or some radiologist just responsible for only 1 or 2 unique abnomalities?

# create dataframe rad_id and annotations (class) they made respectively
data = train_df[["rad_id", "class_id"]]
data = data.value_counts().to_frame().reset_index()
data = data.rename(columns= {0: 'count'})
# sort class in order
data = data.sort_values(by=["class_id"])
# convert to string for showing as legend in graph
data["class_id"] = data["class_id"].astype(str)

In [ ]:
# see each radiologist labeled how many bbox for each class
fig = px.bar(data, x="rad_id", y="count", color="class_id",
             color_discrete_sequence=color_palette_hex,
             title="DISTRIBUTION OF # EACH CLASS ANNOTATION MADE BY EACH RADIOLOGIST"
             ).update_xaxes(categoryorder="total descending")

fig.update_layout(legend_title="CLASS ID",
                  xaxis_title="Radiologist ID",
                  yaxis_title="# annotation in each class")
fig.show()

In [ ]:
# although very least, but some radiologist (except R8, R9, R10) 
# also labeled unique abnomalities
# zoom on those radiologist

# only plot other 14 radiologist except R8, R9, R10
data_zoom = data.drop(data[data["rad_id"].isin(["R8", "R9", "R10"])].index)

fig = px.bar(data_zoom, x="rad_id", y="count", color="class_id",
             color_discrete_sequence=color_palette_hex,
             title="DISTRIBUTION OF # EACH CLASS ANNOTATION MADE BY EACH RADIOLOGIST"
             ).update_xaxes(categoryorder="total descending")

fig.update_layout(legend_title="CLASS ID",
                  yaxis_range=[0, 500],
                  xaxis_title="Radiologist ID (except R8. R9. R10)",
                  yaxis_title="# annotation in each class")
fig.show()

**From the second histogram plotted below we can ascertain the following information**

* Among the other 11 radiologists, 7 of them (R1 through R7) have only ever ()100% annotated images as No finding
* The other 4 radiologists are also heavily skewed towards the No finding label when compared to the main 3 radiologists (R8 through R10). 

4. **BOUNDING BOX EXPLORATION**

In [ ]:
# the bbox coordinates are represented by (xmin, ymin, xmax, ymax).
# visualize the heatmaps to see distribution of each class bboxes
# or the approximate range of locations that the annotations are found 
# and the intensity of the locations within the heatmap.

# get paths to images have bboxes, ignore 'No finding' since they have no bboxes
bbox_df = train_df[train_df["class_id"] != 14].reset_index(drop=True)

In [ ]:
# those dicom have different size in image
# get image size so that we can resize the bboxes in same static size range
# so that we can generate a heatmap that is representative of the actual
# locations of annotations
from tqdm import tqdm

# initialize a dictionary with image name and size respectively
images_size_dict = {}
unique_image_name = bbox_df["image_id"].unique()

for fname in tqdm(unique_image_name, total=len(unique_image_name)):
    path = os.path.join(TRAIN_DIR, fname+".dicom")
    dicom = pydicom.read_file(path)
    images_size_dict[fname] = (dicom.Columns, dicom.Rows)

In [ ]:
# create list of image width and height corresponding to all bbox in dataframe
# not only unique images
image_width, image_height = [], []
for i in bbox_df["image_id"]:
    image_width.append(images_size_dict[i][0])
    image_height.append(images_size_dict[i][1])

# create 2 column width and height fo respective bbox
bbox_df["img_width"] = image_width
bbox_df["img_height"] = image_height

# normalize bbox coordinates in range 0-1
bbox_df["xmin_norm"] = bbox_df["x_min"] / bbox_df["img_width"]
bbox_df["ymin_norm"] = bbox_df["y_min"] / bbox_df["img_height"]
bbox_df["xmax_norm"] = bbox_df["x_max"] / bbox_df["img_width"]
bbox_df["ymax_norm"] = bbox_df["y_max"] / bbox_df["img_height"]

In [ ]:
bbox_df.head()

In [ ]:
# https://www.kaggle.com/craigmthomas/localization-of-findings

# define heatmap size in ration 4:5 as x-ray ratio (width=400px, height=500px)
# image format (width, height), but in nd array format (row=height, col=width)
heatmap_size = (500, 400)

# scale bbox coordinates based on heatmap size, because we will draw bbox in heatmap
bbox_scale = pd.DataFrame()
bbox_scale["class_id"] = bbox_df["class_id"]
bbox_scale["xmin_norm"] = bbox_df["xmin_norm"] * heatmap_size[1]
bbox_scale["ymin_norm"] = bbox_df["ymin_norm"] * heatmap_size[0]
bbox_scale["xmax_norm"] = bbox_df["xmax_norm"] * heatmap_size[1]
bbox_scale["ymax_norm"] = bbox_df["ymax_norm"] * heatmap_size[0]

bbox_scale = bbox_scale.astype(int)
bbox_scale.head()

In [ ]:
# define function draw all bboxes same class on same heatmap
def draw_bbox_on_heatmap(bboxes, class_id):
    # initialize empty (full black) heatmap
    heatmap = np.zeros((heatmap_size))
    for _, row in bboxes[bboxes["class_id"] == class_id].iterrows():
        # draw white bboxes on black heatmap based on bboxes coordinate
        # that mean multiple bboxes which same class_id will be drawn on same heatmap
        heatmap[row[2]:row[4], row[1]:row[3]] += 1
    return heatmap

In [ ]:
# create subplots
fig, axes = plt.subplots(nrows=5, ncols=3, sharey=True, sharex=True, 
                         gridspec_kw={'hspace': .1, 'wspace': 0}, figsize=(12, 26))

for i, ax in enumerate(axes.flatten()):
    # display heatmap
    ax.imshow(draw_bbox_on_heatmap(bbox_scale, i),
              cmap="inferno", interpolation='nearest')
    # set title for heatmap
    _ = ax.set_title(str(i) + ' - ' + class_name[i], size=12)

plt.show()

In [ ]:
# see what % is the bbox area of each class over the image area?
# area of each class bbox = ?% area of image
# because 1 image can contain multiple bboxes for multiple distinct abnomalities
# and they can be overlap together, % area of each class bbox 
# can show impact of that class

bbox_df["bbox_area_norm"] = (bbox_df["xmax_norm"]-bbox_df["xmin_norm"]) \
                          * (bbox_df["ymax_norm"]-bbox_df["ymin_norm"])
bbox_df.head()

In [ ]:
# create custome legend function for visualize
# swap class id to name
def customLegend(fig, nameSwap):
    for i, data in enumerate(fig.data):
        data["name"] = nameSwap[i]
    return fig

In [ ]:
fig = px.box(bbox_df.sort_values(by=["class_id"]), x="class_id", y="bbox_area_norm",
             color="class_id", color_discrete_sequence=color_palette_hex,
             title="DISTRIBUTION OF BBOX AREAS = % SOURCE IMAGE AREA")

fig.update_layout(legend_title="CLASS NAME",
                  yaxis_range=[-0.025,0.4],
                  xaxis_title = "Classes",
                  yaxis_title = "bbox area %")

fig = customLegend(fig, class_name)
fig.show()

# **IMAGE DATA**

In [ ]:
# convert dicom to ndarray function
# from https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way

def dicom_to_array(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.dcmread(path)
    
    # VOI LUT (if available by DICOM device) is used to 
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    
    # make sure the x-ray be visualized in monochorme1, if not x-ray may look inverted
    if fix_monochrome and dicom.PhotometricInterpretation == 'MONOCHROME1':
        # convert background to black (color value 0)
        data = np.max(data) - data
    
    #data = data - np.min(data)
    data = data / np.max(data)                   # normalize in 0-1 range
    data = (data * 255).astype(np.uint8)         # convert in range RGB 0-255
    
    return data

In [ ]:
# create a function receive list of images which already drawn bboxes and plot it
def plot_images(images, images_id,rows=2, cols=2):
    #create subplots
    fig, axs = plt.subplots(rows, cols, figsize=(16, 20), sharex='col', sharey='row',
                            gridspec_kw={'hspace': .05, 'wspace': .05})
    axs = axs.flatten()
    for img, img_id, ax in zip(images, images_id, axs):
        # resize all image to same size
        img = cv2.resize(img, (400, 500))
        ax.set_title("IMAGE ID - " + img_id)
        ax.imshow(img, cmap="gray")
    
    plt.show()

In [ ]:
# create a function draw all bboxes over respective images in given list of image
def draw_bboxes_over_images(images_id_list, draw_all_classes, **kwargs):
    # initialize list of image will be visualized
    IMAGES = []
    
    for image_id in images_id_list:
        path = os.path.join(TRAIN_DIR, image_id+".dicom")
        image = dicom_to_array(path)
        # convert grayscale img (2D) to RBG image (3D)
        image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)

        # get all bboxes of respective image_id and convert to ndarray (.values)
        bboxes = bbox_df.loc[bbox_df["image_id"] == image_id,
                            ["class_id", "x_min", "y_min", "x_max", "y_max"]].astype(int)
        # condition for draw all bboxes (keep all bboxes)
        # or just draw a specific class_id bboxes (keep bboxes of specific class_id)
        if draw_all_classes:
            bboxes = bboxes.values
        else:
            # get optional argument kwargs
            specific_id = kwargs.get('class_id', None)
            bboxes = bboxes[bboxes["class_id"] == specific_id].values

        # draw all bboxes over image
        for box in bboxes:
            class_id = box[0]                            # get class_id 
            class_name = classID_to_STR[class_id]        # get class name from id
            color = color_palette_rbg[class_id]          # get color corresponding id
            
            # draw overlay box (opacity = 0.1) for cooler :))
            alpha = 0.9
            overlay_box = image.copy()
            overlay_box = cv2.rectangle(overlay_box,
                                       (box[1], box[2]), (box[3], box[4]), color, -1)
            image = cv2.addWeighted(image, alpha, overlay_box, 1-alpha, 1.0)
            
            #draw border and add label text
            image = cv2.rectangle(image, (box[1], box[2]), (box[3], box[4]), color, 5)
            image = cv2.putText(image, class_name, (box[1], box[2]-15), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1.75, color, 6)
        
        IMAGES.append(image)
    
    return IMAGES

In [ ]:
import random

# list of unique images which have bboxes
IMAGES_ID = bbox_df["image_id"].unique().tolist()

# choose randome 4 images for visualization
n = random.randint(0, len(IMAGES_ID))
images_id_list = IMAGES_ID[n:n+4]

images_list = draw_bboxes_over_images(images_id_list, draw_all_classes=True)
plot_images(images_list, images_id_list)

# **VISUALIZE EACH ABNOMALITY BBOX**

In [ ]:
# creat a function get a list of 4 images id based on class_id
# and plot them
def plot_specific_class_bboxes(class_id):
    # get images unique id based on given class_id
    IMAGES_ID = bbox_df[bbox_df["class_id"] == class_id]
    IMAGES_ID = IMAGES_ID["image_id"].unique().tolist()
    
    # choose randome 4 images for visualization
    n = random.randint(0, len(IMAGES_ID))
    images_id_list = IMAGES_ID[n:n+4]
    
    # visualize list of images
    images_list = draw_bboxes_over_images(images_id_list, draw_all_classes=False, 
                                          class_id=class_id)
    plot_images(images_list, images_id_list)

0. **AORTIC ENLARGMENT**

In [ ]:
plot_specific_class_bboxes(0)

1. **ATELECTASIS**

In [ ]:
plot_specific_class_bboxes(1)

2. **CALCIFICATION**

In [ ]:
plot_specific_class_bboxes(2)

3. **CARDIOMEGALY**

In [ ]:
plot_specific_class_bboxes(3)

4. **CONSOLIDATION** 

In [ ]:
plot_specific_class_bboxes(4)

5. **ILD**

In [ ]:
plot_specific_class_bboxes(5)

6. **INFILTRATION**

In [ ]:
plot_specific_class_bboxes(6)

7. **LUNG OPACITY** 

In [ ]:
plot_specific_class_bboxes(7)

8. **NODULE/MASS**

In [ ]:
plot_specific_class_bboxes(8)

9. **OTHER LESION**

In [ ]:
plot_specific_class_bboxes(9)

10. **PLEURAL EFFUSION**

In [ ]:
plot_specific_class_bboxes(10)

11. **PLEURAL THICKENING**

In [ ]:
plot_specific_class_bboxes(11)

12. **PNEUMOTHORAX**

In [ ]:
plot_specific_class_bboxes(12)

13. **PULMONARY FIBROSIS**

In [ ]:
plot_specific_class_bboxes(13)

# **FUSING BBOXES**
[Reference](https://www.kaggle.com/sreevishnudamodaran/vinbigdata-fusing-bboxes-coco-dataset)

In [ ]:
# https://github.com/ZFTurbo/Weighted-Boxes-Fusion
!pip install ensemble-boxes

In [ ]:
# import all bboxes fusing methods
from ensemble_boxes import *

In [ ]:
# create function receive a single image and np array of all bboxes
def draw_bboxes_over_single_image(image, bboxes):
    # convert grayscale img (2D) to RBG image (3D)
    image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
        
    # draw all bboxes over image
    for box in bboxes:
        class_id = box[0]                            # get class_id 
        class_name = classID_to_STR[class_id]        # get class name from id
        color = color_palette_rbg[class_id]          # get color corresponding id

        # draw overlay box (opacity = 0.1) for cooler :))
        alpha = 0.9
        overlay_box = image.copy()
        overlay_box = cv2.rectangle(overlay_box, 
                                   (box[1], box[2]), (box[3], box[4]), color, -1)
        image = cv2.addWeighted(image, alpha, overlay_box, 1-alpha, 1.0)

        #draw border and add label text
        image = cv2.rectangle(image, (box[1], box[2]), (box[3], box[4]), color, 5)
        image = cv2.putText(image, class_name, (box[1], box[2]-15), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.75, color, 6)
    
    return image

In [ ]:
from collections import Counter

# create a function receive list of image_id and fusing method
# for each image, we will plot 2 version: original and after apply fusing method
# return a list of image contain both version of each image
def fusing_bboxes(images_id_list, method, iou_thr, skip_box_thr):
    # initialize list of image will be visualized
    IMAGES = []
    IMAGES_ID = []

    for image_id in images_id_list:
        # get image info
        path = os.path.join(TRAIN_DIR, image_id+".dicom")
        image_array = dicom_to_array(path)
        height, width = image_array.shape

        # -------------------- ORIGINAL IMAGE --------------------
        # get all bboxes of respective image_id and convert to ndarray (.values)
        bboxes = bbox_df.loc[bbox_df["image_id"] == image_id, 
                            ["class_id", "x_min", "y_min", "x_max", "y_max"]].astype(int)
        bboxes = bboxes.values

        # plot original image with original bboxes
        image_before = image_array.copy()
        image_before = draw_bboxes_over_single_image(image_before, bboxes)

        IMAGES.append(image_before)
        IMAGES_ID.append(image_id)
        # -------------------- ORIGINAL IMAGE --------------------

        # ------------------- FUSING BBOX IMAGE ------------------
        # create a dictionary count number of bboxes of each class
        count_class = Counter(bboxes[:,0].tolist())
        print(count_class)

        # initialize lists to store class_id and bboxes 
        # which don't have any overlap boxes
        single_class = []
        single_box = []
        # initialize lists to store class_id and bboxes 
        # which have multiple overlap boxes
        multi_classes = []
        multi_boxes = []
        list_scores = []

        unique_classes = np.unique(bboxes[:,0])
        for class_id in unique_classes:
            # select all rows have same class_id (value in column 0)
            specific_class_bboxes = bboxes[np.where(bboxes[:,0]==class_id)]
            # get coordinates of bboxes, take all values except column 0 (class_id)
            bboxes_of_class = specific_class_bboxes[:,1:]

            if count_class[class_id] == 1:
                single_class.append(class_id)
                single_box.append(bboxes_of_class.tolist())
            else:
                # list the class_id, i.g. [3,3,3] or [10,10]
                list_class = specific_class_bboxes[:,0].tolist()
                # set confidence score=1 for all bboxes
                list_scores.append([1 for i in list_class])
                multi_classes.append(list_class)
                # normalize bboxes coordinates to 0-1 for using ensemble-boxes package
                # check data type to use as arguments in ensemble-boxes package
                bboxes_of_class = bboxes_of_class / (width, height, width, height)
                multi_boxes.append(bboxes_of_class.tolist())

        # apply bboxes fusion method
        if method == 'nms':
            print(method)
            boxes, scores, box_labels = nms(multi_boxes, list_scores, 
                                            multi_classes, weights=None, 
                                            iou_thr=iou_thr)
        elif method == 'soft-nms':
            print(method)
            boxes, scores, box_labels = soft_nms(multi_boxes, list_scores, 
                                                 multi_classes, weights=None, 
                                                 iou_thr=iou_thr, thresh=skip_box_thr)
        elif method == 'nmw':
            print(method)
            boxes, scores, box_labels = non_maximum_weighted(multi_boxes, list_scores, 
                                                             multi_classes, weights=None, 
                                                             iou_thr=iou_thr, 
                                                             skip_box_thr=skip_box_thr)
        elif method == 'wbf':
            print(method)
            boxes, scores, box_labels = weighted_boxes_fusion(multi_boxes, list_scores,
                                                              multi_classes, weights=None, 
                                                              iou_thr=iou_thr, 
                                                              skip_box_thr=skip_box_thr)

        # resize bboxes to original size
        boxes = boxes * (width, height, width, height)
        # convert list of single box into np array and stack with bboxes after fusing
        single_box = np.asarray(single_box).reshape((len(single_box),4))
        boxes = np.row_stack((boxes, single_box))

        # stack single class_id np array with class_id after fusing
        single_class = np.asarray(single_class)
        box_labels = np.concatenate([box_labels, single_class])

        # stack label to respective bbox
        boxes = np.column_stack((box_labels, boxes)).astype(int)

        # plot image with bboxes after fusing
        image_after = image_array.copy()
        image_after = draw_bboxes_over_single_image(image_after, boxes)

        IMAGES.append(image_after)
        IMAGES_ID.append(image_id)
        # ------------------- FUSING BBOX IMAGE ------------------
        
    return IMAGES, IMAGES_ID

1. **NON-MAXIMUN SUPPRESSION (NMS)**

In [ ]:
# list of unique images which have bboxes
IMAGES_ID = bbox_df["image_id"].unique().tolist()

# choose randome 2 images for visualization
# before & after fusing bboxes = 4 images
n = random.randint(0, len(IMAGES_ID))
images_id_list = IMAGES_ID[n:n+2]
print(images_id_list)

In [ ]:
# fusing method: NMS
# left column: before fusing
# right column: after fusing
images, images_id = fusing_bboxes(images_id_list, method='nms', 
                                  iou_thr=.5, skip_box_thr=.0001)
plot_images(images, images_id)

2. **SOFT NON-MAXIMUN SUPPRESSION (SOFT-NMS)**

In [ ]:
# fusing method: SOFT-NMS
images, images_id = fusing_bboxes(images_id_list, method='soft-nms', 
                                  iou_thr=.5, skip_box_thr=.0001)
plot_images(images, images_id)

3. **NON-MAXIMUM WEIGHTED (NMW)**

In [ ]:
# fusing method: NMW
images, images_id = fusing_bboxes(images_id_list, method='nmw', 
                                  iou_thr=.5, skip_box_thr=.0001)
plot_images(images, images_id)

4. **WEIGHTED BBOXES FUSION (WBF)**

In [ ]:
# fusing method: WBF
images, images_id = fusing_bboxes(images_id_list, method='wbf', 
                                  iou_thr=.5, skip_box_thr=.0001)
plot_images(images, images_id)